# Extending Plotting

It's time to extend your plotting skills. Over the past two lessons, you've learned how to create a range of interactive plots using hvPlot and Plotly Express; however, you haven't had one centralized location to embed these plots. Now you do! Integrate Plotly map visualizations with hvPlot scatter plots to create a Population and Crimes dashboard.

In [5]:
import plotly.express as px
import panel as pn
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv

### Use extension function to specify plugin

In [6]:
# Set up Panel Plotly extension
pn.extension('plotly')


### Import hvplot.pandas after pn.extension

In [7]:
# Import hvplot.pandas after pn.extension
# This avoids plotly initialization failure

import hvplot.pandas

### Set up Mapbox token and prepare data

In [10]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

# Set token using Plotly Express set function
px.set_mapbox_access_token(map_box_api)

# Read in data
city_pop = pd.read_csv(Path("Resources/Week6-Day3-Activity6-population_counts.csv")).drop_duplicates()
crime_rates = pd.read_csv(Path("Resources/Week6-Day3-Activity6-crime_rates.csv")).drop_duplicates()

pop_with_index = city_pop.set_index("city")
crime_with_index = crime_rates.set_index("city")
population_crime = (
    pd.concat([pop_with_index, crime_with_index], axis=1, sort=True)
    .dropna()
    .reset_index()
)

population_crime.head()

,city,state,latitude,longitude,population_count,pop_2015,violent_crime,murder,murder_rate
0,Albuquerque,NM,35.084491,-106.651137,545852.0,559721.0,5406.0,43.0,7.682399
1,Anaheim,CA,33.835293,-117.914504,336265.0,349471.0,1271.0,18.0,5.150642
2,Anchorage,AK,61.218056,-149.900278,291826.0,301239.0,3226.0,26.0,8.631021
3,Arlington,TX,32.735687,-97.108066,365438.0,387565.0,1946.0,8.0,2.064170
4,Atlanta,GA,33.748995,-84.387982,420003.0,464710.0,5203.0,94.0,20.227669


### Create plots

In [11]:
# Create plots
population_plot = px.scatter_mapbox(
    population_crime,
    lat="latitude",
    lon="longitude",
    size="pop_2015",
    color=population_crime.index,
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="City Population",
    zoom=3,
    width=1000,
)

crime_plot = px.scatter_mapbox(
    population_crime,
    lat="latitude",
    lon="longitude",
    size="violent_crime",
    color=population_crime.index,
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="City Crime",
    zoom=3,
    width=1000,
)

population_violence = population_crime.hvplot.scatter(
    x="pop_2015",
    y="violent_crime",
    title="Violent Crime by Population Correlation",
    width=1000,
).opts(yformatter="%.0f")

violent_murder = population_crime.hvplot.scatter(
    x="violent_crime",
    y="murder",
    title="Correlation Between Number of Violent Crimes and Murder",
    width=1000,
).opts(yformatter="%.0f")

### Create Panel columns and tabs

In [14]:
# Create Panel column for Plotly map plots
geo_column = pn.Column("## Population and Crime Geo Plots", population_plot, crime_plot)

# Create Panel column for hv scatter plots
scatter_column = pn.Column(
    "## Correlation of Population and Crime Plots", population_violence, violent_murder
)
# Add columns to Tab panel
crime_pop_dashboard = pn.Tabs(
    (
        "Geospatial",
        geo_column
    ),
    (
        "Correlations",
        scatter_column
    )
)

crime_pop_dashboard

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Plotly(Figure)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Scatter)
        [2] HoloViews(Scatter)